In [145]:
# trying to setup simple testing square mesh suitable for evolutionary optimization
import gmsh
size = 8  # physical dimension of the side
P = 2     # number of perimeters
width, height = size, size       # setting up the square
num_el_x, num_el_y = size, size  # adjust the num of elements
dx, dy = width / num_el_x, height / num_el_y

# initialize Gmsh
if not gmsh.is_initialized():
    gmsh.initialize()


gmsh.model.add("test_mesh_warwick")

# preparing parameters
lc = 0.5     # mesh size

# defining a namespace
gmg = gmsh.model.geo  # gmg ~ gmsh.model.geo

# defining the points
gmg.addPoint(0, 0, 0, lc, 1)
gmg.addPoint(0, size, 0, lc, 2)
gmg.addPoint(size, size, 0, lc, 3)
gmg.addPoint(size, 0, 0, lc, 4)
gmg.addPoint(0 + P * dx, 0 + P * dy, 0, lc, 5)
gmg.addPoint(0 + P * dx, size - P * dy, 0, lc, 6)
gmg.addPoint(size - P * dx, size - P * dy, 0, lc, 7)
gmg.addPoint(size - P * dx, 0 + P * dy, 0, lc, 8)

gmg.addPoint(P * dx, 0, 0, lc, 9)
gmg.addPoint(P * dx, size, 0, lc, 10)
gmg.addPoint(size - P * dx, size, 0, lc, 11)
gmg.addPoint(size - P * dx, 0 , 0, lc, 12)



# adding lines
gmg.addLine(1, 2, 1)
gmg.addLine(2, 10, 2)
gmg.addLine(10, 11, 3)
gmg.addLine(11, 3, 4)
gmg.addLine(3, 4, 5)
gmg.addLine(4, 12, 6)
gmg.addLine(12, 9, 7)
gmg.addLine(9, 1, 8)
gmg.addLine(5, 6, 9)
gmg.addLine(6, 7, 10)
gmg.addLine(7, 8, 11)
gmg.addLine(8, 5, 12)

gmg.addLine(5, 9, 13)
gmg.addLine(10, 6, 14)
gmg.addLine(11, 7, 15)
gmg.addLine(8, 12, 16)
gmg.addLine(10, 9, 17)   # L inner-side
gmg.addLine(11, 12, 18)  # R inner-side

# defining a new surface:
gmg.addCurveLoop([1, 2, 17, 8], 20)  # left transfinite loop
gmg.addCurveLoop([-14, 3, 15, -10], 21)  # top transfinite loop
gmg.addCurveLoop([-18, 4, 5, 6], 22)  # right transfinite loop
gmg.addCurveLoop([-13, -12, 16, 7], 23)  # bottom transfinite loop
gmg.addCurveLoop([9, 10, 11, 12], 24)  # inner (hole) loop


gmg.addPlaneSurface([20], 1)  # left surface
gmg.addPlaneSurface([21], 2)  # top surface
gmg.addPlaneSurface([22], 3)  # right surface
gmg.addPlaneSurface([23], 4)  # bottom surface
gmg.addPlaneSurface([24], 5)  # inner/infill surface

# synchronize the model
gmg.synchronize()

# adding physical groups
gmsh.model.addPhysicalGroup(1, [1, 2, 3, 4, 5, 6, 7, 8], name="outerLoop") # curves physical group
gmsh.model.addPhysicalGroup(1, [9, 10, 11, 12], name="innerLoop") # curves physical group
gmsh.model.addPhysicalGroup(2, [1, 2, 3, 4], name="PerimeterSurface")   # surfaces physical group


# ftion 'setTransfiniteCurve()' explicitly specifies location of mesh nodes on the curve.
gmg.mesh.setTransfiniteCurve(1, num_el_y + 1)  # uniform points
gmg.mesh.setTransfiniteCurve(17, num_el_y + 1)
gmg.mesh.setTransfiniteCurve(5, num_el_y + 1)
gmg.mesh.setTransfiniteCurve(18, num_el_y + 1)

gmg.mesh.setTransfiniteCurve(3, num_el_x - 2*P + 1)
gmg.mesh.setTransfiniteCurve(10, num_el_x - 2*P + 1)
gmg.mesh.setTransfiniteCurve(7, num_el_x - 2*P + 1)
gmg.mesh.setTransfiniteCurve(12, num_el_x - 2*P + 1)

gmg.mesh.setTransfiniteCurve(2, P + 1)
gmg.mesh.setTransfiniteCurve(8, P + 1)
gmg.mesh.setTransfiniteCurve(6, P + 1)
gmg.mesh.setTransfiniteCurve(4, P + 1)

gmg.mesh.setTransfiniteCurve(14, P + 1)
gmg.mesh.setTransfiniteCurve(15, P + 1)
gmg.mesh.setTransfiniteCurve(13, P + 1)
gmg.mesh.setTransfiniteCurve(16, P + 1)

gmg.mesh.setTransfiniteCurve(9, num_el_y - 2*P + 1)
gmg.mesh.setTransfiniteCurve(11, num_el_y - 2*P + 1)




gmg.synchronize()

# meshing the transfinite surface
gmg.mesh.setTransfiniteSurface(1, "Left", [1, 2, 10, 9])  # set up the corners of interpolation
gmg.mesh.setTransfiniteSurface(2, "Right", [10, 11, 7, 6])  # set up the corners of interpolation
gmg.mesh.setTransfiniteSurface(3, "Right", [11, 3, 4, 12])  # set up the corners of interpolation
gmg.mesh.setTransfiniteSurface(4, "Right", [5, 8, 12, 9])  # set up the corners of interpolation
gmg.mesh.setTransfiniteSurface(5, "Left", [5, 6, 7, 8])  # set up the corners of interpolation


# global meshing options
gmsh.option.setNumber("Mesh.Algorithm", 6)  # Frontal-Delaunay for 2D meshes

# To use quadrilateral mesh elements:
# gmg.mesh.setRecombine(2, 1)
# gmg.mesh.setRecombine(2, 2)
# gmg.mesh.setRecombine(2, 3)
# gmg.mesh.setRecombine(2, 4)

# one more subdivision
#gmsh.option.setNumber("Mesh.SubdivisionAlgorithm", 1)

gmg.synchronize()
gmsh.model.mesh.generate(2)



# Launch the GUI
gmg.synchronize()
gmsh.fltk.run()

# finish the session
gmsh.finalize()


Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 20%] Meshing curve 4 (Line)
Info    : [ 30%] Meshing curve 5 (Line)
Info    : [ 30%] Meshing curve 6 (Line)
Info    : [ 40%] Meshing curve 7 (Line)
Info    : [ 40%] Meshing curve 8 (Line)
Info    : [ 50%] Meshing curve 9 (Line)
Info    : [ 50%] Meshing curve 10 (Line)
Info    : [ 60%] Meshing curve 11 (Line)
Info    : [ 70%] Meshing curve 12 (Line)
Info    : [ 70%] Meshing curve 13 (Line)
Info    : [ 80%] Meshing curve 14 (Line)
Info    : [ 80%] Meshing curve 15 (Line)
Info    : [ 90%] Meshing curve 16 (Line)
Info    : [ 90%] Meshing curve 17 (Line)
Info    : [100%] Meshing curve 18 (Line)
Info    : Done meshing 1D (Wall 0.000714492s, CPU 0.000995s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Transfinite)
Info    : [ 20%] Meshing surface 2 (Transfinite)
Info    : [ 40%] Meshing surface 3 (Transfinite)
Info    : [ 60%